In [1]:
from aiohttp import ClientSession
from typing import Any

In [2]:
import structlog
log = structlog.get_logger()

In [3]:
api_url = 'http://host.docker.internal:8765' # TODO: change for localhost when running outside Docker

async def anki_connect_health_check(session: ClientSession) -> bool:
  response = await session.get(api_url, raise_for_status=True)
  await response.text()
  return True

async def anki_connect_call(session: ClientSession, action: str, **kwargs) -> Any:
  payload = {
    "action": action,
    "params": kwargs,
    "version": 6,
  }
  log.info("anki_connect_call", action=action, params=kwargs)

  response = await session.post(api_url, json=payload)
  data = await response.json()
  if data["error"]:
    raise RuntimeError(data["error"])
  
  return data["result"]

In [4]:
async with ClientSession(raise_for_status=True) as session:
  await anki_connect_health_check(session)
  log.info("anki_connect_health_check", success=True)

2024-05-01 03:24:29 [info     ] anki_connect_health_check      success=True


In [5]:
async with ClientSession(raise_for_status=True) as session:
  deck_names = await anki_connect_call(session, 'deckNames')

deck_names

2024-05-01 03:24:29 [info     ] anki_connect_call              action=deckNames params={}


['Default', 'Designing Data Intensive Applications']

In [6]:
geeder_tag = 'geeder'

In [7]:
deck_name = 'Designing Data Intensive Applications'

async with ClientSession(raise_for_status=True) as session:
  note_ids = await anki_connect_call(session, 'findNotes', query=f'deck:"{deck_name}" tag:{geeder_tag}')

note_ids

2024-05-01 03:24:29 [info     ] anki_connect_call              action=findNotes params={'query': 'deck:"Designing Data Intensive Applications" tag:geeder'}


[1714286765561, 1714286891339]

In [8]:
async with ClientSession(raise_for_status=True) as session:
  notes = await anki_connect_call(session, 'notesInfo', notes=note_ids)

notes

2024-05-01 03:24:30 [info     ] anki_connect_call              action=notesInfo params={'notes': [1714286765561, 1714286891339]}


[{'noteId': 1714286765561,
  'tags': ['geeder'],
  'fields': {'Front': {'value': 'What is head-of-line blocking?', 'order': 0},
   'Back': {'value': '<div><div><div><div><div><div><div><div>Head-of-line (HOL) blocking occurs when a single slow or blocked transaction or connection in a queue prevents all subsequent transactions or connections from being processed or transmitted, thereby degrading the overall performance of the system or network.</div></div></div></div></div></div></div></div>',
    'order': 1}},
  'modelName': 'Basic',
  'cards': [1714286765561]},
 {'noteId': 1714286891339,
  'tags': ['geeder'],
  'fields': {'Front': {'value': 'What is tail latency&nbsp;amplification?',
    'order': 0},
   'Back': {'value': '<div><div><div><div><div><div><div><div>Tail latency amplification occurs when a high-latency event or outlier in one component of a distributed system propagates and significantly amplifies the overall response time for the entire system.</div></div></div></div></d